In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Eddie Cochran,I've Waited So Long,"You are my loved one, you are my all \r\nCome..."
1,Omd,Shame,If there's one thing I know \r\nThings are go...
2,Linkin Park,Forgotten,From the top to the bottom \r\nAt the core I'...
3,LL Cool J,I Wish (Remix),[Ll Cool J](Puffy) \r\nIts gonna be a hot sum...
4,Imagine Dragons,The River,In the latest midnight hour \r\nWhen the worl...
5,George Jones,Angels Don't Fly,Deep into midnight far from the warm light \r...
6,Pretenders,Dragway 42,I walk the streets at night \r\nUnder artific...
7,Dolly Parton,Deportee,The crops are all in and the peaches are rotti...
8,Jackson Browne,You've Forgotten,You've been passed around from hand to hand \...
9,Phil Collins,Take Me With You,If you see the light on in the window of my ro...


In [9]:
df['text'][0]

"You are my loved one, you are my all  \r\nCome to my arms and love me forever  \r\nI've waited so long  \r\nYou are the reason for bluebird songs  \r\nYou are my loved one, you are the reason  \r\nI've waited so long  \r\nTime never mattered till you left me in vain  \r\nMy heart was shattered, put it together and then  \r\nHeaven will love us, surely you know  \r\nThat stardust will twinkle above us  \r\nI've waited so long\r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.09247075, 0.02296645, ..., 0.09508326, 0.0586186 ,
       0.05935467])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))